# Getting all coins from Messari API

In [28]:
# libraries
import json
import requests
import os
import utils as u

## Getting all the tokens from the all time high endpoint

In [2]:
# URL and headers for the Messari API call
url = 'https://api.messari.io/marketdata/v1/assets/ath'

# API key from environment variable
API_KEY = os.getenv('MESSARI_ANDRE_KEY')

headers = {
    'accept': 'application/json',
    'x-messari-api-key': API_KEY,
}

In [3]:
response = requests.get(url, headers=headers)

In [6]:
if response.status_code == 200:
    result = response.json()

    data = result['data']

    with open('../data/raw/all-tokens.json', 'w') as json_file:
        json.dump(data, json_file, indent=4)

    print(f"Information saved in '../data/raw/all-tokens.json'. Number of tokens: {len(data)}")
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Information saved in '../data/raw/all-tokens.json'. Number of tokens: 14728


## Filtering the coins

Since we have the tokens organized by the objective categories in `logical-model/data/csv`, wee need to get rid of the ones with market cap 0.

In [7]:
# libraries
import pandas as pd

In [20]:
ai_df = pd.read_csv('../../logical-model/data/csv/coins_AI.csv')
gaming_df = pd.read_csv('../../logical-model/data/csv/coins_gaming.csv')
meme_df = pd.read_csv('../../logical-model/data/csv/coins_meme.csv')
rwa_df = pd.read_csv('../../logical-model/data/csv/coins_RWA.csv')

print("Shapes of the dataframes:", ai_df.shape, gaming_df.shape, meme_df.shape, rwa_df.shape)
print("Number of tokens: ", ai_df.shape[0] + gaming_df.shape[0] + meme_df.shape[0] + rwa_df.shape[0])

Shapes of the dataframes: (250, 17) (400, 17) (500, 17) (100, 17)
Number of tokens:  1250


### Preprocessing data before filtering

In [21]:
def remove_non_market_cap(df: pd.DataFrame) -> pd.DataFrame:
    df_without_null = df.dropna(subset=['market_cap'])
    return df_without_null[df_without_null['market_cap'] != 0]

In [22]:
ai_df = remove_non_market_cap(ai_df)
gaming_df = remove_non_market_cap(gaming_df)
meme_df = remove_non_market_cap(meme_df)
rwa_df = remove_non_market_cap(rwa_df)

print("Shapes of the dataframes:", ai_df.shape, gaming_df.shape, meme_df.shape, rwa_df.shape)
print("Number of tokens: ", ai_df.shape[0] + gaming_df.shape[0] + meme_df.shape[0] + rwa_df.shape[0])

Shapes of the dataframes: (167, 17) (261, 17) (220, 17) (75, 17)
Number of tokens:  723


### Filtering by category

In [29]:
# read the json file with all tokens
tokens = u.loadJSON('../data/raw/all-tokens.json')
len(tokens)

14728

In [38]:
def filter_by_category(category_df: pd.DataFrame, tokens: list) -> pd.DataFrame:
    return [
        record for record in tokens
        if (record['name'] in category_df['name'].values) and
        # (record['slug'] in category_df['slug'].values) and
        (record['symbol'] in category_df['symbol'].astype(str).values)
    ]

In [40]:
meme = filter_by_category(category_df=meme_df, tokens=tokens)
gaming = filter_by_category(category_df=gaming_df, tokens=tokens)
ai = filter_by_category(category_df=ai_df, tokens=tokens)
rwa = filter_by_category(category_df=rwa_df, tokens=tokens)

In [41]:
meme

[{'id': '02e9c2cc-2e3b-45fe-b7bb-508cb23a3a39',
  'serialId': 260634,
  'symbol': 'PEPE',
  'name': 'Pepe',
  'slug': 'pepe-0',
  'contractAddresses': [{'platform': 'ethereum',
    'contractAddress': '0x6982508145454ce325ddbe47a25d4ec3d2311933'},
   {'platform': 'arbitrum-one',
    'contractAddress': '0x25d887ce7a35172c62febfd67a1856f20faebb00'},
   {'platform': 'binance-smart-chain',
    'contractAddress': '0x25d887ce7a35172c62febfd67a1856f20faebb00'}],
  'allTimeHighData': {'price': 1.6993576632582742e-05,
   'at': '2024-05-27T15:00:00Z',
   'daysSince': 140.4457350402865,
   'percentDown': 36.100516020005145,
   'breakevenMultiple': 1.5649578646254358},
  'cycleLowData': {'price': 6.140886029402947e-06,
   'at': '2024-08-05T12:00:00Z',
   'percentUp': 76.82802979851289,
   'daysSince': 70.57073504028651}},
 {'id': '05bf91a3-52e5-40ef-b09d-0c471617116e',
  'serialId': 7472,
  'symbol': 'CATGIRL',
  'name': 'Catgirl',
  'slug': 'catgirl',
  'contractAddresses': [{'platform': 'binance-